In [1]:
# from huggingface_hub import login
# login()   # then paste your access token from https://huggingface.co/settings/tokens
!pip install -U "transformers>=4.30.0"
!nvidia-smi
!pip install torch
!pip install -U "transformers[torch]"
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
import pandas as pd
import re
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
import pandas as pd
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os

Mon Nov  3 22:48:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 576.83                 Driver Version: 576.83         CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   58C    P8              4W /   45W |       0MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

C:\Users\chock\miniconda3\envs\is5126\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Login using e.g. `huggingface-cli login` to access this dataset
# splits = {'train': 'train.json', 'test': 'test.json'}
# train_df = pd.read_json("hf://datasets/jason23322/high-accuracy-email-classifier/" + splits["train"])
# test_df = pd.read_json("hf://datasets/jason23322/high-accuracy-email-classifier/" + splits["test"])
train_df = pd.read_csv('combined dataset/combined_train.csv')
test_df = pd.read_csv('combined dataset/combined_test.csv')
# View categories
print(train_df['category'].value_counts())
print(test_df['category'].value_counts())

category
forum                1800
verify_code          1800
social_media         1796
promotions           1796
spam                 1794
updates              1794
flight_booking        960
concert_promotion     960
Name: count, dtype: int64
category
verify_code          451
forum                450
social_media         449
updates              449
promotions           449
spam                 449
flight_booking       240
concert_promotion    240
Name: count, dtype: int64


In [3]:
train_df

,id,subject,body,text,category,category_id
0,promotions_582,Anniversary Special: Buy one get one free,"As our loyal customer, get exclusive $60 off $...",Anniversary Special: Buy one get one free As o...,promotions,1
1,spam_1629,Your Amazon was used on new device,Your $5000 refund is processed. Claim: bit.ly/...,Your Amazon was used on new device Your $5000 ...,spam,3
2,spam_322,Re: Your Google inquiry,"Hi, following up about your Google application...","Re: Your Google inquiry Hi, following up about...",spam,3
3,social_media_80,Digital Ritual Experience Creation,Cross-cultural ceremony design. Join: virtualr...,Digital Ritual Experience Creation Cross-cultu...,social_media,2
4,forum_1351,"Your post was moved to ""Programming Help""","Trending: ""cooking"" (258 comments). View: supp...","Your post was moved to ""Programming Help"" Tren...",forum,0
...,...,...,...,...,...,...
12695,music_415,🤘 The Ultimate Calvin Harris Experience for th...,Calvin Harris is back! Catch them live at Heng...,🤘 The Ultimate Calvin Harris Experience for th...,concert_promotion,6
12696,flight_1177,Group Booking Confirmed: Cathay Pacific CA636,Explore New York on 2025-11-12! Flight CA636 w...,Group Booking Confirmed: Cathay Pacific CA636 ...,flight_booking,7
12697,music_168,🤘 The Ultimate Die Twice Experience for the t...,Early bird tickets end soon! Die Twice at Quee...,🤘 The Ultimate Die Twice Experience for the t...,concert_promotion,6
12698,music_546,❤️ Get Ready for Prateek Kuhad on 06-09-2024 –...,"Good news, Melbourne! Prateek Kuhad is perform...",❤️ Get Ready for Prateek Kuhad on 06-09-2024 –...,concert_promotion,6


In [4]:
test_df

,id,subject,body,text,category,category_id
0,social_media_1558,Watch later: Recommended story,"Group update: ""Book Club"" posted video. Trendi...","Watch later: Recommended story Group update: ""...",social_media,2
1,social_media_505,News from groups you follow,"Group ""Tech Enthusiasts"" invited you. RSVP: pl...","News from groups you follow Group ""Tech Enthus...",social_media,2
2,forum_190,Two-Factor Authentication Enforcement Notice,Required for all accounts by Dec 1: security.f...,Two-Factor Authentication Enforcement Notice R...,forum,0
3,updates_1851,Security upgrade: 2FA enabled,Your monthly statement is available. View/down...,Security upgrade: 2FA enabled Your monthly sta...,updates,4
4,verify_code_1753,Verification PIN: 907472,Use 404583 as your verification code. Device: ...,Verification PIN: 907472 Use 404583 as your ve...,verify_code,5
...,...,...,...,...,...,...
3172,flight_812,Luxury Travel Awaits! Singapore Airlines SI554...,Adventure awaits! Singapore Airlines Flight SI...,Luxury Travel Awaits! Singapore Airlines SI554...,flight_booking,7
3173,music_441,❤️ The Ultimate twenty one pilots Experience f...,"Feel the bass, live the moment. twenty one pil...",❤️ The Ultimate twenty one pilots Experience f...,concert_promotion,6
3174,flight_11,Your Journey with IndiGo is About to Begin!,Your journey is ready! Flight IN818 with IndiG...,Your Journey with IndiGo is About to Begin! Yo...,flight_booking,7
3175,music_207,🎶 Experience Lizzo performing hits like ['Ever...,Experience the incredible stage presence of Li...,🎶 Experience Lizzo performing hits like ['Ever...,concert_promotion,6


In [5]:
os.environ["WANDB_DISABLED"] = "true"

# --- Helper preprocessing function ---
def preprocess_text(text):
    """Preprocess text exactly as done during training"""
    text = text.lower()
    text = re.sub(r'http[s]?://\S+', 'URL', text)
    text = re.sub(r'www\.\S+', 'URL', text)
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', 'EMAIL', text)
    text = re.sub(r'\b\d+\b', 'NUMBER', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = ' '.join(text.split())
    return text


# --- Load dataset ---
# splits = {'train': 'train.json', 'test': 'test.json'}
# train_df = pd.read_json("hf://datasets/jason23322/high-accuracy-email-classifier/" + splits["train"])
# test_df = pd.read_json("hf://datasets/jason23322/high-accuracy-email-classifier/" + splits["test"])

# Use preprocessed text and numeric labels
train_df['text'] = train_df['text'].apply(preprocess_text)
test_df['text'] = test_df['text'].apply(preprocess_text)
train_df = train_df.rename(columns={'category_id': 'label'})
test_df = test_df.rename(columns={'category_id': 'label'})


# --- Dataset class for PyTorch ---
class EmailDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=128):
        self.texts = df['text'].tolist()
        self.labels = df['label'].tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        )
        item = {key: val.squeeze(0) for key, val in encoding.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item


# --- Tokenizer and Datasets ---
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_dataset = EmailDataset(train_df, tokenizer)
test_dataset = EmailDataset(test_df, tokenizer)

# --- Model setup ---
num_labels = train_df['label'].nunique()
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)


# --- Metrics ---
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1': f1}


# --- Training configuration ---
training_args = TrainingArguments(
    output_dir='./results',
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=200,
    save_steps=500,
    save_total_limit=2
)



# --- Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


# --- Train and evaluate ---
trainer.train()
# trainer.evaluate()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
C:\Users\chock\AppData\Local\Temp\ipykernel_15076\2877793780.py:92: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
200,0.472500
400,0.044400
600,0.018000
800,0.012300
1000,0.003700


TrainOutput(global_step=1191, training_loss=0.09312657700377688, metrics={'train_runtime': 862.6303, 'train_samples_per_second': 44.167, 'train_steps_per_second': 1.381, 'total_flos': 2506267811635200.0, 'train_loss': 0.09312657700377688, 'epoch': 3.0})

In [6]:
# Evaluate the model
metrics = trainer.evaluate(eval_dataset=test_dataset)

# Print metrics nicely
print("=== Test Set Evaluation ===")
for key, value in metrics.items():
    if isinstance(value, float):
        print(f"{key}: {value:.4f}")
    else:
        print(f"{key}: {value}")


=== Test Set Evaluation ===
eval_loss: 0.0165
eval_accuracy: 0.9972
eval_precision: 0.9972
eval_recall: 0.9972
eval_f1: 0.9972
eval_runtime: 28.0014
eval_samples_per_second: 113.4590
eval_steps_per_second: 3.5710
epoch: 3.0000


In [7]:
import torch
import torch.nn.functional as F

# --- Dynamically build category mapping from dataset ---
# Make sure `train_df` contains 'label' and 'category'
category_id_to_name = dict(zip(train_df['label'].unique(), train_df['category'].unique()))
print(category_id_to_name) 
# Example output: {0: 'Forum', 1: 'Promotions', 2: 'Social Media', 3: 'Spam', 4: 'Updates', 5: 'Verify Code'}

def predict_email_category(email_text, model, tokenizer, max_len=128):
    """
    Predict the category of a single email text.

    Returns:
        dict: {
            'predicted_category': str,
            'confidence': float,
            'all_probabilities': dict
        }
    """
    # Preprocess
    text = preprocess_text(email_text)

    # Tokenize
    encoding = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=max_len,
        return_tensors='pt'
    )

    # Move to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    # Forward pass
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = F.softmax(logits, dim=-1).squeeze().cpu().numpy()

    # Prediction
    pred_id = int(probs.argmax())
    predicted_category = category_id_to_name.get(pred_id, f"unknown_{pred_id}")
    confidence = float(probs[pred_id])

    all_probabilities = {category_id_to_name.get(i, f"unknown_{i}"): float(probs[i]) for i in range(len(probs))}

    return {
        'predicted_category': predicted_category,
        'confidence': confidence,
        'all_probabilities': all_probabilities
    }

# --- Example usage ---
email_text = "Your verification code is 123456. Please enter this code."
result = predict_email_category(email_text, model, tokenizer)
print("Category:", result['predicted_category'])       # 🔐 Verify Code
print("Confidence:", result['confidence'])
print("All probabilities:", result['all_probabilities'])


{np.int64(1): 'promotions', np.int64(3): 'spam', np.int64(2): 'social_media', np.int64(0): 'forum', np.int64(5): 'verify_code', np.int64(4): 'updates', np.int64(7): 'flight_booking', np.int64(6): 'concert_promotion'}
Category: verify_code
Confidence: 0.9979040622711182
All probabilities: {'forum': 0.00030554417753592134, 'promotions': 0.0002425546117592603, 'social_media': 0.0002409506414551288, 'spam': 0.000224732983042486, 'updates': 0.00026603107107803226, 'verify_code': 0.9979040622711182, 'concert_promotion': 0.00033322584931738675, 'flight_booking': 0.00048291345592588186}


In [8]:
examples = [
    "Your verification code is 123456. Please enter this code.",  # Verify Code
    "Big sale! Get 50% off all items today only.",                # Promotions
    "Mike Chen commented on your post in Programming Help.",      # Social Media
    "Your post was moved to 'Cooking' forum.",                    # Forum
    "Security alert: 5 failed login attempts detected.",          # Spam
    "Your August system update is now available."                 # Updates
    """

    Subject: model bookin

    Dear Alex,

Thank you for booking with Delta Airlines.
Your flight DL-342 from New York (JFK) to Los Angeles (LAX) departs on Nov 12, 2025, at 8:45 AM.

Check-in opens 24 hours before departure. You can manage your booking at www.delta.com
.

Safe travels,
The Delta Airlines Team"""
]

for email_text in examples:
    result = predict_email_category(email_text, model, tokenizer)
    print("Email text:", email_text)
    print("Predicted Category:", result['predicted_category'])
    print("Confidence:", round(result['confidence'], 3))
    print("All probabilities:", {k: round(v,3) for k,v in result['all_probabilities'].items()})
    print("-"*80)


Email text: Your verification code is 123456. Please enter this code.
Predicted Category: verify_code
Confidence: 0.998
All probabilities: {'forum': 0.0, 'promotions': 0.0, 'social_media': 0.0, 'spam': 0.0, 'updates': 0.0, 'verify_code': 0.998, 'concert_promotion': 0.0, 'flight_booking': 0.0}
--------------------------------------------------------------------------------
Email text: Big sale! Get 50% off all items today only.
Predicted Category: promotions
Confidence: 0.998
All probabilities: {'forum': 0.0, 'promotions': 0.998, 'social_media': 0.0, 'spam': 0.0, 'updates': 0.0, 'verify_code': 0.0, 'concert_promotion': 0.0, 'flight_booking': 0.0}
--------------------------------------------------------------------------------
Email text: Mike Chen commented on your post in Programming Help.
Predicted Category: social_media
Confidence: 0.986
All probabilities: {'forum': 0.007, 'promotions': 0.0, 'social_media': 0.986, 'spam': 0.001, 'updates': 0.004, 'verify_code': 0.0, 'concert_promotio

In [9]:
train_df

,id,subject,body,text,category,label
0,promotions_582,Anniversary Special: Buy one get one free,"As our loyal customer, get exclusive $60 off $...",anniversary special buy one get one free as ou...,promotions,1
1,spam_1629,Your Amazon was used on new device,Your $5000 refund is processed. Claim: bit.ly/...,your amazon was used on new device your NUMBER...,spam,3
2,spam_322,Re: Your Google inquiry,"Hi, following up about your Google application...",re your google inquiry hi following up about y...,spam,3
3,social_media_80,Digital Ritual Experience Creation,Cross-cultural ceremony design. Join: virtualr...,digital ritual experience creation cross cultu...,social_media,2
4,forum_1351,"Your post was moved to ""Programming Help""","Trending: ""cooking"" (258 comments). View: supp...",your post was moved to programming help trendi...,forum,0
...,...,...,...,...,...,...
12695,music_415,🤘 The Ultimate Calvin Harris Experience for th...,Calvin Harris is back! Catch them live at Heng...,the ultimate calvin harris experience for the ...,concert_promotion,6
12696,flight_1177,Group Booking Confirmed: Cathay Pacific CA636,Explore New York on 2025-11-12! Flight CA636 w...,group booking confirmed cathay pacific ca636 e...,flight_booking,7
12697,music_168,🤘 The Ultimate Die Twice Experience for the t...,Early bird tickets end soon! Die Twice at Quee...,the ultimate die twice experience for the tour...,concert_promotion,6
12698,music_546,❤️ Get Ready for Prateek Kuhad on 06-09-2024 –...,"Good news, Melbourne! Prateek Kuhad is perform...",get ready for prateek kuhad on NUMBER NUMBER N...,concert_promotion,6


In [11]:
import json
# --- Paths ---
save_dir = "./artifacts"

# --- Assume `model` is your trained BERT and `tokenizer` is your tokenizer ---

# --- Build label mappings from your training DataFrame ---
# train_df must have 'label' (numeric) and 'category' (string)
category_id_to_name = train_df.drop_duplicates(subset=['label', 'category']).set_index('label')['category'].to_dict()
category_name_to_id = {v: int(k) for k, v in category_id_to_name.items()}

# --- Inject mappings into model config ---
model.config.id2label = {int(k): str(v) for k, v in category_id_to_name.items()}
model.config.label2id = {str(v): int(k) for k, v in category_id_to_name.items()}

# --- Save model and tokenizer ---
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

# --- Save mappings to JSON ---
with open(f"{save_dir}/label_mappings.json", "w") as f:
    json.dump({
        "label2id": model.config.label2id,
        "id2label": model.config.id2label
    }, f, indent=2)

print(f"✅ Model, tokenizer, and label mappings saved to {save_dir}")


✅ Model, tokenizer, and label mappings saved to ./artifacts


In [12]:
# import torch
# import json
# from transformers import BertTokenizer, BertForSequenceClassification

# # --- Load model and tokenizer ---
# model_path = "./saved_model"
# tokenizer = BertTokenizer.from_pretrained(model_path)
# model = BertForSequenceClassification.from_pretrained(model_path)
# model.eval()  # disable dropout etc.

# # --- Load label mappings ---
# try:
#     with open(f"{model_path}/label_mappings.json", "r") as f:
#         mappings = json.load(f)
#         id2label = {int(k): v for k, v in mappings["id2label"].items()}
# except FileNotFoundError:
#     print("⚠️ Warning: label_mappings.json not found — using config labels instead.")
#     id2label = {int(k): v for k, v in model.config.id2label.items()}

# # --- Inference function ---
# def predict(text):
#     """Classify a single text string."""
#     inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
#     with torch.no_grad():
#         outputs = model(**inputs)
#         pred_id = torch.argmax(outputs.logits, dim=1).item()
#         label = id2label.get(pred_id, str(pred_id))
#     return label

# # --- Example predictions ---
# examples = [
#     "Your flight from Paris to New York has been confirmed.",
#     "You’ve won a free vacation! Click here to claim your tickets.",
#     "Can we move our meeting to Monday?",
#     """

#     Subject: model bookin

#     Dear Alex,

# Thank you for booking with Delta Airlines.
# Your flight DL-342 from New York (JFK) to Los Angeles (LAX) departs on Nov 12, 2025, at 8:45 AM.

# Check-in opens 24 hours before departure. You can manage your booking at www.delta.com
# .

# Safe travels,
# The Delta Airlines Team"""
# ]

# for text in examples:
#     label = predict(text)
#     print(f"Text: {text}\nPredicted label: {label}\n")


In [13]:
# import re
# import torch
# import numpy as np
# from transformers import BertTokenizer, BertForSequenceClassification

# class EmailClassifierPipeline:
#     def __init__(self, model_path="artifacts/email_classifier"):
#         self.model_path = model_path
#         self.model = BertForSequenceClassification.from_pretrained(model_path)
#         self.tokenizer = BertTokenizer.from_pretrained(model_path)
#         self.model.eval()

#         self.label_map = {
#             0: "forum",
#             1: "promotions",
#             2: "social_media",
#             3: "spam",
#             4: "updates",
#             5: "verify_code",
#             6: "concert_promotion",
#             7: "flight_booking"
#         }


#     def preprocess_text(self, text):
#         """Same preprocessing as training"""
#         text = text.lower()
#         text = re.sub(r'http[s]?://\S+', 'URL', text)
#         text = re.sub(r'www\.\S+', 'URL', text)
#         text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', 'EMAIL', text)
#         text = re.sub(r'\b\d+\b', 'NUMBER', text)
#         text = re.sub(r'[^\w\s]', ' ', text)
#         text = ' '.join(text.split())
#         return text

#     def predict(self, texts):
#         """Predict label and probability for one or multiple texts"""
#         if isinstance(texts, str):
#             texts = [texts]

#         # Preprocess
#         texts = [self.preprocess_text(t) for t in texts]
#         inputs = self.tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=128)

#         with torch.no_grad():
#             outputs = self.model(**inputs)
#             probs = torch.nn.functional.softmax(outputs.logits, dim=1).numpy()

#         preds = np.argmax(probs, axis=1)
#         readable_labels = [self.label_map.get(p, str(p)) for p in preds]

#         return readable_labels, probs


ModuleNotFoundError: No module named 'email_pipeline'